# Baseline Yolo8n wandb

In this Notebook the Baseline Model will be trained with sweeps  
Please preregister on the homepage and provide the necessary API Key in the .env file  
Maybee using Sweeps could ease the process

In [1]:
# check the .env file 
from dotenv import load_dotenv
import os

# Load the .env file
load_dotenv()

# Get and print the WANDB_API_KEY
wandb_api_key = os.getenv("WANDB_API_KEY")
print(f"WANDB_API_KEY: [{wandb_api_key[:4]}...]")

WANDB_API_KEY: [69ca...]


In [3]:
import wandb
from wandb.integration.ultralytics import add_wandb_callback
import keras
import random
import numpy as np

from pathlib import Path
from ultralytics import YOLO

wandb: WARNING This integration is tested and supported for ultralytics v8.0.238 and below.
wandb: WARNING             Please report any issues to https://github.com/wandb/wandb/issues with the tag `yolov8`.


In [4]:
import os
from pathlib import Path

PROJECT_ROOT = Path("/home/jovyan/DSPRO2/M-AI-ZE-Maize-diseases-detection")

os.environ["WANDB_DIR"] = str(PROJECT_ROOT / "wandb")

YOLO_OUTPUT_DIR = str(PROJECT_ROOT / "runs")

print("WANDB_DIR set to:", os.environ["WANDB_DIR"])
print("YOLO_OUTPUT_DIR will be used in model.train(project=...)")


WANDB_DIR set to: /home/jovyan/DSPRO2/M-AI-ZE-Maize-diseases-detection/wandb
YOLO_OUTPUT_DIR will be used in model.train(project=...)


## Sweep Training for Baseline


In [19]:
dataset_yaml = "/exchange/dspro2/M-AI-ZE/data/adjusted/1.1/splits/SID01/data.yaml"

TypeError: add_wandb_callback() missing 1 required positional argument: 'model'

In [13]:
sweep_config = {
    "method": "grid",
    "name": "yolov8_sweep4",
    "metric": {
        "name": "metrics/mAP50",
        "goal": "maximize"
    },
    "parameters": {
        "epochs": {"values": [30, 50]},
        "imgsz": {"values": [640]},
        "batch": {"values": [16]},
        "lr0": {"values": [0.001, 0.005, 0.01]},
        "model": {"values": ["yolov8n.pt"]}
    }
}

In [14]:
def train_yolo_sweep():
    run = wandb.init(
        project="maize_detection",
        name=f"sweep_yolov8n_{wandb.util.generate_id()}",
        group="yolov8n_sweep",
        tags=["yolov8n", "sweep", "baseline"],
    )
    config = run.config
    model = YOLO(config.model)
    add_wandb_callback(model, enable_model_checkpointing=True)
    model.train(
        data=dataset_yaml,
        epochs=wandb.config.epochs,
        imgsz=wandb.config.imgsz,
        batch=wandb.config.batch,
        lr0=wandb.config.lr0,
        project=YOLO_OUTPUT_DIR,
        name=f"sweep_yolov8n_{run.id}",
        exist_ok=True
    )
    
    source_path = Path(YOLO_OUTPUT_DIR) / "detect" / f"sweep_yolov8n_{run.id}" / "weights" / "best.pt"
    target_path = Path(PROJECT_ROOT) / "model" / f"best_{run.id}.pt"
    target_path.parent.mkdir(parents=True, exist_ok=True)

    if source_path.exists():
        shutil.copy(source_path, target_path)
        print(f"Best model saved to: {target_path}")
    else:
        print(f"Best model not found at: {source_path}")
        
    run.finish()


In [15]:
sweep_id = wandb.sweep(sweep=sweep_config, project="maize_detection")
wandb.agent(sweep_id, function=train_yolo_sweep, count=6)


Create sweep with ID: 59nghszq
Sweep URL: https://wandb.ai/rueedi-tobias-hochschule-luzern/maize_detection/sweeps/59nghszq


wandb: Agent Starting Run: 60ypya8g with config:
wandb: 	batch: 16
wandb: 	epochs: 30
wandb: 	imgsz: 640
wandb: 	lr0: 0.001
wandb: 	model: yolov8n.pt
wandb: ERROR Failed to detect the name of this notebook. You can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


wandb: 
wandb: 🚀 View run sweep_yolov8n_3hozxy8k at: https://wandb.ai/rueedi-tobias-hochschule-luzern/maize_detection/runs/jyyp7gom
wandb: Find logs at: ../wandb/wandb/run-20250422_123931-jyyp7gom/logs


Ultralytics 8.3.113 🚀 Python-3.12.10 torch-2.6.0+cu124 CUDA:0 (NVIDIA A16, 14891MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/exchange/dspro2/M-AI-ZE/data/adjusted/1.1/splits/SID01/data.yaml, epochs=30, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=/home/jovyan/DSPRO2/M-AI-ZE-Maize-diseases-detection/runs, name=sweep_yolov8n_60ypya8g, exist_ok=True, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, sa

train: Scanning /exchange/dspro2/M-AI-ZE/data/adjusted/1.1/splits/SID01/labels/train.cache... 10858 images, 0 backgrounds, 0 corrupt: 100%|██████████| 10858/10858 [00:00<?, ?it/s]


...

                   all       1357      10668      0.554      0.457       0.47      0.214


Generating Visualizations for batch-1/1:   0%|          | 0/32 [00:00<?, ?it/s]

Generating Visualizations for batch-1/1:   0%|          | 0/32 [00:00<?, ?it/s]


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       6/30      3.64G      1.947      1.684       1.39        201        640:  82%|████████▏ | 555/679 [02:04<00:30,  4.03it/s]wandb: Ctrl + C detected. Stopping sweep.
       6/30      3.64G      1.948      1.683       1.39        171        640:  84%|████████▍ | 572/679 [02:08<00:25,  4.19it/s]wandb: ERROR Problem finishing run
Exception in thread Thread-24 (_run_job):
Traceback (most recent call last):
  File "/opt/conda/lib/python3.12/site-packages/torch/utils/data/dataloader.py", line 1251, in _try_get_data
    data = self._data_queue.get(timeout=timeout)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.12/queue.py", line 179, in get
    raise Empty
_queue.Empty

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/opt/conda/lib/python3.12/site-packages/tqdm/std.py", line 1181, in __iter__
    for obj in iterable:
               ^^^^^^^^
  File "/opt/conda/lib/python3.12/site-packages/ultr